<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Разобьём-данные-на-выборки" data-toc-modified-id="Разобьём-данные-на-выборки-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Разобьём данные на выборки</a></span></li><li><span><a href="#Исследуйте-модели" data-toc-modified-id="Исследуйте-модели-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследуйте модели</a></span><ul class="toc-item"><li><span><a href="#Решающее-дерево" data-toc-modified-id="Решающее-дерево-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Решающее дерево</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Выводы</a></span></li></ul></li><li><span><a href="#Проверьте-модели-на-тестовой-выборке" data-toc-modified-id="Проверьте-модели-на-тестовой-выборке-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Проверьте модели на тестовой выборке</a></span></li></ul></div>

# Построение моделей для рекомендации тарифов 

Оператор мобильной связи хочет построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».


У нас есть данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф с максимально большим значением accuracy (>0.75). 

Предобработка данных не понадобится.

Откроем файл с данными и изучим его. 
Разделим исходные данные на обучающую, валидационную и тестовую выборки.
Исследуем качество разных моделей, меняя гиперпараметры.
Проверим качество модели на тестовой выборке.

In [35]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier # чтобы обучать модель для классификации
from sklearn.metrics import accuracy_score  # чтобы оценивать качество моделей
from sklearn.ensemble import RandomForestClassifier # алгоритм случайного леса
from sklearn.model_selection import train_test_split # чтобы делить выборки
from sklearn.linear_model import LogisticRegression # алгоритм логистической регрессии
from sklearn.dummy import DummyClassifier # создаёт простую модель для проверки на адекватность

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

Посмотрим на первые десять строк таблицы:

In [3]:
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
- `сalls` — количество звонков,
- `minutes` — суммарная длительность звонков в минутах,
- `messages` — количество sms-сообщений,
- `mb_used` — израсходованный интернет-трафик в Мб,
- `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [4]:
df['is_ultra'].mean()

0.30647168637212197

In [5]:
df['is_ultra'].count()

3214

Десятую часть датасета составляют пользователи тарифа ультра. Остальные объекты принадлежат тарифу смарт.

## Разобьём данные на выборки

Все объекты из датафрейма необходимо разделить на три выборки: тренировочную, валидационную и тестовую.

На тренировочной модели мы будем обучать модель, на вадидационной проверять работоспособность модели (выявлять переобучение, например), а на тестовой её тестировать. Разобъём данные на выборки в соотношении 3:1:1.

Первым шагом разобъём данные на целевой признак (тариф, которым пользуется человек) и признаки, которые подвели человека к использованию этого тарифа.

In [6]:
features = df.drop(['is_ultra'], axis=1)

In [7]:
target = df['is_ultra'].copy()

Разделим данные на две выборки: данные, необходимые до тестирования и данные для тестов. Потом данные, нужные до тестирования разделим на тренировочные и валидационные.

In [8]:
 # отделим 20% данных для тестовой выборки
features_before_test, features_test, target_before_test, target_test = train_test_split(features, target, test_size=0.2, random_state=12345).copy()

In [9]:
 # отделим 25% данных для валидационной выборки
features_train, features_valid, target_train, target_valid = train_test_split(features_before_test, target_before_test, test_size=0.25, random_state=12345).copy()

Разделили объекты на три выборки, положили их в отдельные переменные:
- features_train, target_train это выборки для тренировки модели,
-  features_valid, target_valid - выборки для валидации модели,
- features_test, target_test - выборки для тестирования.

Проверим размеры полученых наборов, размеры валидационной и тестовой выборки должны совпасть:

In [10]:
features_train.shape

(1928, 4)

In [11]:
 target_train.shape

(1928,)

In [12]:
features_valid.shape

(643, 4)

In [13]:
 target_valid.shape

(643,)

In [14]:
features_test.shape

(643, 4)

In [15]:
target_test.shape

(643,)

## Исследование моделей

### Решающее дерево

Построим и обучим модель на основе решающего дерева. 

Посмотрим на параметр accuray, чтобы понять, на сколько модель промахнулась. Аccuray это тношение числа правильных ответов к размеру тестовой выборки — «доля правильных ответов». 


Чтобы получить максимально высокое качество предсказания нужно перебрать в алгоритме обучения разные варианты глубины дерева.
Найдём модель с самой большой долей правильных ответов. Будем рассматривать глубину дерева от 1 до 6.

In [16]:
best_model_tree = None
best_result = 0
best_depth = None
for depth in range(1, 7):
  model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
  model.fit(features_train, target_train) # обучим модель

  train_predictions = model.predict(features_train) # получим предсказания модели
  valid_predictions = model.predict(features_valid) 
  
  result_train = accuracy_score(target_train, train_predictions) # посчитаем качество модели
  result_valid = accuracy_score(target_valid, valid_predictions)

  print (f'Учебная выборка: {result_train}, валидационная выборка: {result_valid}, глубина дерева: {depth}')
  if result_valid > best_result:
    best_model_tree = model
    best_result = result_valid
    best_depth = depth
        
print("Accuracy лучшей модели:", best_result)
print("Лучшая глубина дерева", best_depth)

Учебная выборка: 0.758298755186722, валидационная выборка: 0.7387247278382582, глубина дерева: 1
Учебная выборка: 0.79201244813278, валидационная выборка: 0.7573872472783826, глубина дерева: 2
Учебная выборка: 0.8117219917012448, валидационная выборка: 0.7651632970451011, глубина дерева: 3
Учебная выборка: 0.8205394190871369, валидационная выборка: 0.7636080870917574, глубина дерева: 4
Учебная выборка: 0.8272821576763485, валидационная выборка: 0.7589424572317263, глубина дерева: 5
Учебная выборка: 0.8335062240663901, валидационная выборка: 0.7573872472783826, глубина дерева: 6
Accuracy лучшей модели: 0.7651632970451011
Лучшая глубина дерева 3


Можно сказать, что модель переобучилась, поскольку accuray этой модели на тренировочной выборке больше.

### Случайный лес

Попробуем другой алгоритм классификации — случайный лес. Алгоритм обучает большое количество независимых друг от друга деревьев, а потом принимает решение на основе голосования. Случайный лес помогает улучшить результат предсказания и избежать переобучения.

In [17]:
best_model_forest = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(10, 51, 10):
  for depth in range (1, 11):
    model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth) # создадим модель с заданным количеством деревьев и глубиной дерева
    model.fit(features_train, target_train) # обучим модель на тренировочной выборке
    result = model.score(features_valid,target_valid) # посчитаем качество модели на валидационной выборке
    if result > best_result:
        best_model_forest = model # сохраним наилучшую модель
        best_result = result #  сохраним наилучшее значение метрики accuracy на валидационных данных
        best_est = est
        best_depth = depth

print(f"Accuracy наилучшей модели на валидационной выборке: {best_result}, Количество деревьев: {best_est}, Максимальная глубина: {best_depth}")


Accuracy наилучшей модели на валидационной выборке: 0.7978227060653188, Количество деревьев: 50, Максимальная глубина: 10


Если сделать гиперпараметр n_estimators больше, модель начнёт разрастаться и медленно обучаться.

### Логистическая регрессия

Алгоритмы не ограничиваются деревьями. Есть и другие способы классификации.

инициализируем модель логистической регрессии с параметром random_state=12345

In [18]:
model_lin_reg = LogisticRegression(random_state=12345) 

 Обучим модель на тренировочной выборке

In [19]:
model_lin_reg.fit(features_train,target_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=12345, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

Получим метрику качества модели на валидационной выборке:


In [20]:
result = model_lin_reg.score(features_valid,target_valid)
result

0.6967340590979783

Получилось не очень много, зато быстро.

### Выводы




Оценим модели по качеству (accuracy) и скорости работы.
1. Качество (accuracy). Это важный критерий: чем выше качество, тем больше прибыли приносит продукт.
Самое высокое качество у случайного леса: вместо одного решающего дерева используется целый лес.
На втором месте — дерево решений. При глубине дерева равной 3 модель дала хороший результат.
Самое низкое качество предсказания у логистической регрессии. Модель несложная, но с этой задачей она справилась плохо.
2. Скорость работы тоже значимый критерий.
Высокая скорость работы у логистической регрессии: у неё меньше всего параметров.
Скорость решающего дерева тоже высокая и зависит от глубины. Наша модель получила ответ на вопрос всего за три проверки значений признаков — это очень быстро. Случайный лес медленнее всех: чем больше деревьев, тем медленнее работает модель.


## Проверьте модели на тестовой выборке





Протестируем:

In [21]:
test_predictions  = best_model_tree.predict(features_test)
accuracy_score(target_test, test_predictions)

0.7869362363919129

In [22]:

best_model_forest.score(features_test,target_test)

0.7993779160186625

In [23]:
model_lin_reg.fit(features_test,target_test)
model_lin_reg.score(features_test,target_test)

0.702954898911353

У всех моделей на тестовой выборке значение accuray больше 0,75.

Посмотрим, что предскажет простейшая модель, которая предсказывает только наиболее встречающийся класс.

In [34]:
dummy_clf = DummyClassifier(strategy="most_frequent", random_state=12345)
dummy_clf.fit(features_train, target_train)
dummy_clf.score(features_test, target_test)

0.6951788491446346

Простейшая модель ошибается в 30% случаев, а хорошо обученные модели в 20% случаев. Значит, модели адекватно обучились.